<a href="https://colab.research.google.com/github/shhrrtnvr/rank_scrapper/blob/main/uri_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import ssl
import re
import csv
import itertools
import pandas as pd

In [2]:
def create_context():
  ctx = ssl.create_default_context()
  ctx.check_hostname = False
  ctx.verify_mode = ssl.CERT_NONE
  return ctx

In [3]:
def scrap(url):
  headers={'User-Agent': 'Mozilla/5.0'}
  req = Request(url, headers = headers)
  html = urlopen(req, context=create_context())
  return BeautifulSoup(html, 'html.parser')


In [4]:
def getNameAndSolved(user):
  columns = list(user.find_all('td'))
  try:
    name, solved = columns[2].a.get_text(), columns[3].get_text()
    solved = int(re.findall('[0-9]+', solved)[0])
    return name, solved
  except:
    return None


In [5]:
def make_table(soup):
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  return rows


In [6]:
def filter_table(table):
    filtered_table = list()
    for row in table:
        ret = getNameAndSolved(row)
        if ret:
            filtered_table.append(ret)
    return filtered_table

In [7]:
url = 'https://www.urionlinejudge.com.br/judge/en/users/university/{uni}?page={page}&direction=DESC'

In [8]:
data = list()
p = itertools.count(1)
while True:
  page = url.format(uni='ulab', page=next(p))
  try:
    soup = scrap(page)
    table = make_table(soup)
    table = filter_table(table)
    data.extend(table)
  except:
    break


In [9]:
print(len(data))

105


In [10]:
data.sort(key=lambda x: x[1], reverse=True)

In [11]:
print(data)

[('Mohd Ataul Hakim', 119), ('Apon Ghosh', 111), ('Sadia Ahmed Bushra', 100), ('Rakib Ahmed', 83), ('yyyyy', 67), ('Nur Hasan Masum', 62), ('Bambi_18', 62), ('Razaan Reza', 60), ('Jenifar Gomes', 59), ('plabon', 56), ('ulab_dilware', 55), ('Mohammad Eunus', 54), ('Nahian Raisa', 51), ('Farjahan Akter Boby', 49), ('Md. Rahimuzzaman Bhuiyan', 47), ('ABRAR MAHBUB TANIM (201014073)', 42), ('Arunangshu Mojumder Raatul', 41), ('Munem Shariar', 41), ('D4RK_H4T (ULAB)', 40), ('IrtezaAsad', 38), ('Rukon Uddin', 34), ('Mizanur Rahman', 31), ('Tony Nicholas Gomes', 31), ('Ahmed Moonwar Anjum', 31), ('Abel', 29), ('152014012', 28), ('Hajera Khatun Saba', 26), ('A Z M Shamsuddin', 26), ('Md Sazzadul Islam Shykat', 25), ('Ehasan', 23), ('Kaniz Amena Anjona', 23), ('Afraim Zahangir', 22), ('anjon mahmud', 21), ('Muhesena Nasiha Syeda', 21), ('Sazzad Hussain', 21), ('wahidullah remon', 20), ('Sadia Afrin Ridita', 20), ('Souvik Ahmed', 19), ('Kazi Nafisa', 19), ('152014046', 19), ('Morium Begum (201014

In [12]:
names, count = [a[0] for a in data], [a[1] for a in data]

In [13]:
df = pd.DataFrame({
    'Name': names,
    'Sovle Count':count
})

In [14]:
pd.set_option('display.max_rows', None)
df


,Name,Sovle Count
0,Mohd Ataul Hakim,119
1,Apon Ghosh,111
2,Sadia Ahmed Bushra,100
3,Rakib Ahmed,83
4,yyyyy,67
5,Nur Hasan Masum,62
6,Bambi_18,62
7,Razaan Reza,60
8,Jenifar Gomes,59
9,plabon,56
